# 計算機程式期末報告-貓狗分辨

## 組員名單:
統計二 107304006 呂嘉恩

統計二 107304009 吳松霖

統計二 107304033 蔡睿承

財政二 107205018 黃國洋

## 專案分工:

統計二 107304006 呂嘉恩：資料搜尋，建模及找出較好的數字

統計二 107304009 吳松霖：資料集的分類整理、建模，打註解

統計二 107304033 蔡睿承：資料集分類整理、打亂，Debug，打註解

財政二 107205018 黃國洋：資料搜尋，建模及找出最好的優化器

由於是大家聚在一起做的，全員都有參與到，所以其實很難細分

## 問題發想:

我們好奇同為長身體四隻腳兩耳朵，體態相近的貓跟狗，僅有些許五官面孔的不同，

AI是否能夠準確的辨認出來。

因此採用外型相同，五官輪廓較明顯不同的貓和狗，作為訓練資料，應該較不易有問題。

## 流程介紹:

這次總共呈現了兩個模型，分別是模型一及模型二。

模型一為我們較早期做出的模型，模型一共用了三種不同的最佳化辦法，取了準確率最高的Adam作為優化器的選擇。

但我們後來在進行優化調整時，發現由於經驗不足，而沒有將資料集打亂再進行訓練。

而我們用的分類方式又沒辦法很好的將其打亂，因此模型二我們採用了不同的資料分類方式。而準確率有些許提升。

#### 簡易流程如下：

#### 模型一

1.載入模組

2.匯入資料集並整理

3.打造CNN模型

4.訓練資料

5.預測

#### 模型二

1.載入模組

2.匯入資料集並將其打亂

3.將打亂資料集做整理

4.打造CNN模型

5.訓練資料

6.預測

## 資料來源:

Kaggle:'https://www.kaggle.com/c/dogs-vs-cats'

# 模型一

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
import os #處理文件用
import cv2 #處理影像

In [2]:
print(os.listdir()) #確認我們目錄下的資料夾名稱

['.git', '.ipynb_checkpoints', 'cat.jpg', 'cat2.jpg', 'CNN_Dog_vs_Cat  (Best) - 複製 - 複製 (2).ipynb', 'CNN_Dog_vs_Cat  (Best) - 複製.ipynb', 'CNN_Dog_vs_Cat  (Best).ipynb', 'CNN_Dog_vs_Cat 2 (Best).ipynb', 'dog.jpg', 'dog2.jpg', 'example.png', 'homework5.ipynb', 'hw001.ipynb', 'hw002.ipynb', 'hw003.ipynb', 'hw004.ipynb', 'hw04.ipynb', 'hw1-national index.ipynb', 'hw2-Daily Foreign Investor Buying Rank.ipynb', 'hw3.ipynb', 'hw6 rnn.ipynb', 'myCNNmodel.h5', 'new test.ipynb', 'news.ipynb', 'order.ipynb', 'pandas test.ipynb', 'sampleSubmission.csv', 'test', 'test1', 'train', 'train1', 'transfer-learning-hw.ipynb', 'Untitled Diagram (1).png', 'Untitled.ipynb', '問題回答.ipynb', '期末function api hw.ipynb']


### 導入Kaggle上下載的資料集，並整理資料

In [3]:
x_train=[] #創建空的list
y_train=[]
path=os.path.join('train1') #在這裡令一個路徑，這樣要換資料夾時才不用去程式內部一個一個改

for i in os.listdir(path):
    name=i.split(".")[0]   #找出圖片以'.'為切割的名字開頭
#     print(name)
    if name=="dog":        #如果圖片名稱為dog開頭則 label=1，反之若是cat則 label=0
        label=int(1)
    else:
        label=int(0)
#     print(label)
    img=cv2.imread(os.path.join(path,i))
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     print(cv2.imread(os.path.join(path,i)))
    nimg=cv2.resize(img,dsize=(150,150))    #將每個圖片大小調整成150X150
    x_train.append(nimg) #將調整好的圖片加入list
    y_train.append(label) #加入它們的label編號


### 資料處理

In [ ]:
x_train=np.array(x_train).reshape(-1,150,150,3) #重塑資料，用-1讓他自己去判斷個數
x_train=x_train/255 #標準化

In [ ]:
x_train.shape #25000筆,150x150的照片,灰階圖片所以1個Channel

In [ ]:
x_train[5].shape

In [ ]:
y_train= np.array(y_train) #將資料轉成array
y_train[5] 

### 打造我們的CNN

In [ ]:
#命名一個新模型
model = Sequential() 

#第一層Filter用3x3的矩陣，數量為32，池化窗口為2x2，激活函數為relu
model.add(Conv2D(32,(3,3),input_shape=(150,150,3),padding='same',activation ='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

#第二層Filter用3x3的矩陣，數量為64，池化窗口為2x2，激活函數為relu
model.add(Conv2D(64,(3,3),padding='same',activation ='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten()) #拉平

#第一層為64個神經元，激活函數為relu
model.add(Dense(64, activation='relu'))

#第二層為1個神經元，因為資料為二分類所以最後一層激活函數使用sigmoid
model.add(Dense(1, activation='sigmoid'))


### 檢視模型

In [ ]:
model.summary()

### 在這裡我們optimizer總共試了3種，Adam為準確率最高的0.82

### SGD(learning rate使用0.01) 準確率為0.6多

### AdaGrad 準確率則為0.72上下

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### 訓練資料，每100筆資料修正一次，跑5次

In [ ]:
model.fit(x_train,y_train, epochs=3, batch_size=500,validation_split = (1/6))

### 準確率希望有8成，因此在多訓練1次

In [ ]:
# model.fit(x_train,y_train, epochs=1, batch_size=100)

In [ ]:
train_acc = model.history.history['accuracy']
valid_acc = model.history.history['val_accuracy']
plt.plot(train_acc , label='accuracy')
plt.plot(valid_acc, label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')


In [ ]:
train_loss = model.history.history['loss']
valid_loss = model.history.history['val_loss']
plt.plot(train_loss, label='loss')
plt.plot(valid_loss, label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0, 1])
plt.legend(loc='upper right')

### 導入測試資料並整理

In [ ]:
x_test=[] #建立空的list
path=os.path.join("test1") #設定資料夾

for i in os.listdir(path):
    img=cv2.imread(os.path.join(path,i)) #使用灰階，並將圖片轉為array態
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    nimg=cv2.resize(img,dsize=(150,150)) #將圖片大小改為150x150
    x_test.append(nimg) #加入圖片資料進list


In [ ]:
x_test=np.array(x_test).reshape(-1,150,150,3) #重塑維度
x_test=x_test/255 #標準化

### 預測

In [ ]:
result = model.predict_classes(x_test)

In [ ]:
def my_predict(n):
    if result[n]==int(0):   #label=0 則為貓
        a='貓'
    else:
        a='狗'              #label=1 則為狗
    print('我可愛的 CNN 預測是', a)
    X = x_test[n].reshape(150,150)
    
    plt.imshow(X)

In [ ]:
my_predict(87)

### BUTBUTBUT (心路歷程可跳過)

#### 身為Python小白的我們四個發現一件事情

#### 我們做的模型，少做一件事情，是在做資料學習時很重要的事

#### 也就是打亂資料集

#### 上網爬了很多文章發現如果沒有打亂資料集，有可能會影響模型的泛化能力

#### 進而導致我們耳熟的overfitting狀況

#### 於是乎在經過漫長的爬文時間後，終於看懂了怎麼打亂資料集

#### 於是，再參考許多資料後發現了這篇，有一步步的教學，雖然下面幾乎看不懂，


#### 但是這篇打亂的地方還是能勉強看懂的

#### "https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/491275/#outline__1_2"

#### 搭配著print學習法，不斷把資料print出來後我們終於知道要怎麼打亂資料集了

# 模型二

### 我們首先一樣導入train的資料，但因為要打亂


### 我們模型一命名label的方式要做修改

### 不然打亂的label沒辦法跟著它的圖片一致

### 一樣將檔名為 cat 開頭為 label=0，dog 則 label=1

In [ ]:
path='train1' #設定路徑
cats =  [] #建立空list
label_cats = []
dogs = []
label_dogs = []
for file in os.listdir(path):
    name = file.split(sep='.') #以.為分割
    if name[0]=='cat':
        cats.append(file) #將檔名加進list
        label_cats.append(0) #貓的label=0
    else:
        dogs.append(file)
        label_dogs.append(1) #狗的label=1



### 檢視資料我們發現前面圖片均為貓，後面圖片均為狗

In [ ]:
print(cats,label_cats)
print(dogs,label_dogs)

### 為了要解決這個問題我們決定將資料集順序打亂

### 首先我們將前面貓與狗的list結合

In [ ]:
image_list = np.hstack((cats, dogs))#將貓狗圖片堆積起來
label_list = np.hstack((label_cats,label_dogs))#label也堆積起來

In [ ]:
print(image_list) #檢視
print(label_list)

### 結合後轉為array，並將其轉置

In [ ]:
temp = np.array([image_list,label_list])

In [ ]:
print(temp)#檢視

In [ ]:
temp = temp.transpose()#轉置矩陣

In [ ]:
print(temp)#檢視轉置後結果

### 接下來使用shuffle打亂順序

In [ ]:
np.random.shuffle(temp)#打亂矩陣順序

In [ ]:
print(temp)#檢視

### 再將打亂後的資料提取出

In [ ]:
image_list = list(temp[:,0])#提取矩陣第一行

In [ ]:
print(image_list)#檢視

In [ ]:
label_list = list(temp[:,1])#提取矩陣第二行

In [ ]:
print(label_list)#檢視

In [ ]:
label_list = [int(i) for i in label_list]#將文字轉成數字

In [ ]:
print(label_list)#檢視

### 成功提取資料後，我們看到資料已經被打亂

### 因此利用opencv修改圖片大小為150x150，並將其轉成array

In [ ]:
x_train=[] #建立空的list
y_train=label_list 
path=os.path.join('train1') #設定路徑
for j in image_list:
    img=cv2.imread(os.path.join(path,j))
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    nimg=cv2.resize(img,dsize=(150,150)) #修改大小
    x_train.append(nimg) #加入array型態的圖片資料

In [ ]:
# print(x_train)#檢視

In [ ]:
# print(y_train)#檢視

### 重塑資料將其轉成我們要的

In [ ]:
x_train=np.array(x_train).reshape(-1,150,150,3) #轉換資料
x_train=x_train/255 #標準化

In [ ]:
x_train.shape #25000筆,150x150的照片,1個Channel灰階

In [ ]:
x_train[5].shape #確認資料格式

In [ ]:
print(x_train)#檢視

In [ ]:
y_train= np.array(y_train) #將list轉成array,並確認資料格式
y_train[5]

In [ ]:
print(y_train)#檢視

### 資料處理完畢後

### 套用我們第一次使用的模型

In [ ]:
#命名一個新模型
model = Sequential() 

#第一層Filter用3x3的矩陣，數量為32，池化窗口為2x2，激活函數為relu
model.add(Conv2D(32,(3,3),input_shape=(150,150,3),padding='same',activation ='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

#第二層Filter用3x3的矩陣，數量為64，池化窗口為2x2，激活函數為relu
model.add(Conv2D(64,(3,3),padding='same',activation ='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten()) #拉平

#第一層為64個神經元，激活函數為relu
model.add(Dense(64, activation='relu'))

#第二層為1個神經元，因為資料為二分類所以最後一層激活函數使用sigmoid
model.add(Dense(1, activation='sigmoid'))


### 檢視模型

In [ ]:
model.summary()

### compile模型

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### 訓練模型，模型一訓練4次準確率0.82，模型二打亂後準確率提高

In [ ]:
model.fit(x_train,y_train, epochs=5, batch_size=100,validation_split = (1/6))

In [ ]:
train_acc = model.history.history['accuracy']
valid_acc = model.history.history['val_accuracy']
plt.plot(train_acc , label='train_acc')
plt.plot(valid_acc, label = 'valid_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
train_loss = model.history.history['loss']
valid_loss = model.history.history['val_loss']
plt.plot(train_loss, label='train_loss')
plt.plot(valid_loss, label = 'valid_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0, 1])
plt.legend(loc='upper right')

### 導入我們的測試資料並整理

In [ ]:
x_test=[] #建立空的list
path=os.path.join("test1") #設定資料夾

for i in os.listdir(path):
    img=cv2.imread(os.path.join(path,i)) #使用灰階，並將圖片轉為array態
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    nimg=cv2.resize(img,dsize=(150,150)) #將圖片大小改為150x150
    x_test.append(nimg) #加入圖片資料進list


In [ ]:
x_test=np.array(x_test).reshape(-1,150,150,3) #重塑資料
x_test=x_test/255 #標準化

### 預測

In [ ]:
result = model.predict_classes(x_test)

In [ ]:
def my_predict(n):
    if result[n]==int(0):
        a='貓'
    else:
        a='狗'
    print('我可愛的 CNN 預測是', a)
    X = x_test[n].reshape(150,150,3)
    plt.imshow(X)

In [ ]:
my_predict(1)